# 🚀 First Spark Application

**Phase 1 (Fundamentals) - Your First Spark Program**

**Prerequisites**: Basic Scala knowledge, Docker/Spark installation

**Estimated time**: 30-45 minutes

---

## 🎯 Learning Goals

By the end of this notebook, you'll be able to:
- ✅ Create and configure a SparkSession
- ✅ Work with Resilient Distributed Datasets (RDDs)
- ✅ Perform basic transformations and actions
- ✅ Understand lazy evaluation in Spark
- ✅ Build a simple word count application
- ✅ Monitor Spark applications

---

## 📋 Table of Contents

1. [SparkSession Setup](#setup)
2. [Basic RDD Operations](#rdd)
3. [Transformations vs Actions](#transform)
4. [Word Count Application](#wordcount)
5. [Monitoring & Debugging](#monitoring)
6. [Exercises](#exercises)
7. [Next Steps](#next)

---

## ⚙️ SparkSession Setup

**SparkSession is the entry point for all Spark functionality.**

In [ ]:
// Import Spark libraries
import org.apache.spark.sql.SparkSession
import org.apache.spark.rdd.RDD

In [ ]:
// Create SparkSession
val spark = SparkSession.builder()
  .appName("First Spark Application")
  .master("local[*]")  // Use all available cores
  .getOrCreate()

// Get SparkContext (for RDD operations)
val sc = spark.sparkContext

println("🚀 Spark Application Started!")
println(s"🔥 Spark Version: ${spark.version}")
println(s"🎯 Master: ${sc.master}")
println(s"⚡ Available Cores: ${sc.defaultParallelism}")
println(s"📊 Application ID: ${sc.applicationId}")

**What just happened?**

- **`SparkSession.builder()`**: Creates a builder for SparkSession
- **`.appName()`**: Sets application name (shows in UI)
- **`.master()`**: Specifies cluster manager (`local[*]` = local mode)
- **`.getOrCreate()`**: Gets existing session or creates new one
- **`spark.sparkContext`**: Access to lower-level RDD API

**Key Points:**
- Always create SparkSession at the beginning
- `local[*]` uses all available CPU cores
- Spark UI will be available at http://localhost:4040

## 🔢 Basic RDD Operations

**RDD (Resilient Distributed Dataset) is Spark's core abstraction for distributed data.**

In [ ]:
// Create RDD from a collection
val numbers = Seq(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
val numbersRDD = sc.parallelize(numbers)

println("📊 Original Data:")
println(s"Numbers: ${numbers.mkString(", ")}")
println(s"RDD partitions: ${numbersRDD.getNumPartitions}")

In [ ]:
// Basic RDD operations
println("🔄 RDD Transformations:")

// Filter even numbers
val evenNumbers = numbersRDD.filter(_ % 2 == 0)
println(s"Even numbers: ${evenNumbers.collect().mkString(", ")}")

// Square all numbers
val squaredNumbers = numbersRDD.map(x => x * x)
println(s"Squared numbers: ${squaredNumbers.collect().mkString(", ")}")

// Filter numbers greater than 5, then double them
val processedNumbers = numbersRDD
  .filter(_ > 5)
  .map(_ * 2)
println(s"Numbers > 5, doubled: ${processedNumbers.collect().mkString(", ")}")

In [ ]:
// Aggregation operations (Actions)
println("📈 RDD Actions:")
println(s"Count: ${numbersRDD.count()}")
println(s"Sum: ${numbersRDD.sum()}")
println(f"Average: ${numbersRDD.mean()}%.2f")
println(s"Min: ${numbersRDD.min()}")
println(s"Max: ${numbersRDD.max()}")
println(s"First 3 elements: ${numbersRDD.take(3).mkString(", ")}")
println(s"Sample (20%): ${numbersRDD.takeSample(false, 2).mkString(", ")}")

## 🔄 Transformations vs Actions

**Understanding lazy evaluation is crucial in Spark!**

In [ ]:
// Demonstrate lazy evaluation
println("🦥 Lazy Transformations (no computation yet):")

val lazyRDD = numbersRDD
  .filter(_ > 5)
  .map(_ * 2)
  .filter(_ < 18)

println("✓ Transformations defined but not executed")
println(s"RDD type: ${lazyRDD.getClass.getSimpleName}")
println(s"RDD partitions: ${lazyRDD.getNumPartitions}")

// Now trigger computation with an action
println("\n⚡ Action Triggers Computation:")
val result = lazyRDD.collect()
println(s"Result: ${result.mkString(", ")}")
println("✓ Computation completed!")

In [ ]:
// Chain multiple operations
println("🔗 Operation Chaining:")

val chainResult = numbersRDD
  .filter(_ % 2 == 0)        // Keep even numbers
  .map(_ * _ * _)            // Cube them
  .filter(_ > 100)           // Keep > 100
  .map(math.sqrt(_).toInt)   // Take square root
  .collect()

println(s"Chained operations result: ${chainResult.mkString(", ")}")
println("\n💡 Spark optimizes the entire pipeline before execution!")

## 📝 Word Count Application

**The "Hello World" of big data - counting words in text.**

In [ ]:
// Sample text data (simulating multiple documents)
val textData = Seq(
  "Hello Spark World Hello",
  "Spark is awesome for big data",
  "Hello world of distributed computing",
  "Spark makes big data processing easy",
  "Hello Spark and Scala world"
)

println("📄 Sample Text Data:")
textData.zipWithIndex.foreach { case (text, idx) =>
  println(f"${idx + 1}. $text")
}

In [ ]:
// Classic word count implementation
println("📊 Word Count Implementation:")

val wordCountRDD = sc.parallelize(textData)
  .flatMap(line => line.toLowerCase.split("\\s+"))  // Split into words
  .filter(_.nonEmpty)                                // Remove empty strings
  .map(word => (word, 1))                           // Create (word, 1) pairs
  .reduceByKey(_ + _)                               // Sum counts by word
  .sortBy(_._2, ascending = false)                  // Sort by frequency

println("Word frequencies:")
println("=" * 30)
wordCountRDD.collect().foreach { case (word, count) =>
  println(f"  $word%-12s: $count")
}

In [ ]:
// Word count with additional analytics
println("📈 Advanced Word Count Analytics:")

val wordStats = wordCountRDD.map { case (word, count) =>
  (word, count, word.length)
}

// Most frequent words
val topWords = wordCountRDD.take(5)
println("\n🏆 Top 5 most frequent words:")
topWords.foreach { case (word, count) =>
  println(f"  '$word' appears $count times")
}

// Words with frequency > 1
val repeatedWords = wordCountRDD.filter(_._2 > 1).collect()
println(s"\n🔄 Words appearing more than once: ${repeatedWords.length}")
repeatedWords.foreach { case (word, count) =>
  println(f"  '$word': $count times")
}

// Total statistics
val totalWords = wordCountRDD.map(_._2).sum()
val uniqueWords = wordCountRDD.count()
val avgFrequency = totalWords.toDouble / uniqueWords

println(f"\n📊 Statistics:")
println(f"  Total words: $totalWords")
println(f"  Unique words: $uniqueWords")
println(f"  Average frequency: $avgFrequency%.2f")

## 🔍 Monitoring & Debugging

**Spark provides excellent tools for monitoring and debugging applications.**

In [ ]:
// Application information
println("📊 Application Information:")
println(s"Application Name: ${spark.sparkContext.appName}")
println(s"Application ID: ${spark.sparkContext.applicationId}")
println(s"Spark Version: ${spark.version}")
println(s"Scala Version: ${scala.util.Properties.versionString}")
println(s"Master URL: ${spark.sparkContext.master}")
println(s"Default Parallelism: ${spark.sparkContext.defaultParallelism}")
println(s"Available Cores: ${Runtime.getRuntime.availableProcessors}")

In [ ]:
// RDD lineage and debugging
println("🔗 RDD Lineage (for debugging):")

val debugRDD = sc.parallelize(1 to 10)
  .filter(_ > 5)
  .map(_ * 2)
  .filter(_ < 18)

println("RDD Debug Info:")
println(s"  Class: ${debugRDD.getClass.getSimpleName}")
println(s"  Partitions: ${debugRDD.getNumPartitions}")
println(s"  Dependencies: ${debugRDD.dependencies.length}")

// Show actual lineage
println("\nRDD Lineage:")
println(debugRDD.toDebugString)

In [ ]:
// Performance monitoring
println("⚡ Performance Monitoring:")

val startTime = System.nanoTime()

// Perform some operations
val performanceTest = sc.parallelize(1 to 10000)
  .filter(_ % 2 == 0)
  .map(_ * _)
  .sum()

val endTime = System.nanoTime()
val durationMs = (endTime - startTime) / 1e6

println(f"Performance test completed in $durationMs%.2f ms")
println(f"Result: $performanceTest")

// Memory information
val runtime = Runtime.getRuntime
val mb = 1024 * 1024
println(f"\n🧠 Memory Usage:")
println(f"  Total: ${runtime.totalMemory() / mb} MB")
println(f"  Free: ${runtime.freeMemory() / mb} MB")
println(f"  Used: ${(runtime.totalMemory() - runtime.freeMemory()) / mb} MB")

## 🏆 Exercises

**Test your understanding with these hands-on exercises.**

In [ ]:
// Exercise 1: Temperature Analysis
// Given a list of temperatures, calculate:
// - Average temperature
// - Number of days above average
// - Hottest and coldest days

val temperatures = Seq(22.5, 25.0, 19.8, 28.3, 24.7, 21.1, 26.9, 23.4)

// FIXME: Implement temperature analysis
val avgTemp = ???  // Calculate average
val aboveAvg = ??? // Count days above average
val hottest = ???  // Find hottest temperature
val coldest = ???  // Find coldest temperature

println("🌡️ Temperature Analysis:")
println(f"Average temperature: $avgTemp%.1f°C")
println(f"Days above average: $aboveAvg")
println(f"Hottest day: $hottest%.1f°C")
println(f"Coldest day: $coldest%.1f°C")

In [ ]:
// Exercise 2: Text Analytics
// Analyze the text data for:
// - Total number of words
// - Average words per line
// - Most common starting word
// - Lines containing "Spark"

// FIXME: Implement text analytics
val totalWords = ???     // Total word count
val avgWordsPerLine = ??? // Average words per line
val sparkLines = ???     // Lines containing "Spark"

println("📝 Text Analytics:")
println(s"Total words: $totalWords")
println(f"Average words per line: $avgWordsPerLine%.1f")
println(s"Lines containing 'Spark': ${sparkLines.length}")
sparkLines.foreach(line => println(s"  $line"))

In [ ]:
// Exercise 3: Number Crunching
// Work with numbers 1-100:
// - Sum of all numbers
// - Count of even numbers
// - Product of numbers divisible by 7
// - Numbers that are both even and divisible by 3

val numbers = 1 to 100

// FIXME: Implement number crunching
val sumAll = ???           // Sum of all numbers
val evenCount = ???        // Count of even numbers
val sevenProduct = ???     // Product of numbers divisible by 7
val evenAndDivBy3 = ???    // Numbers that are even AND divisible by 3

println("🔢 Number Crunching (1-100):")
println(s"Sum of all numbers: $sumAll")
println(s"Even numbers count: $evenCount")
println(s"Product of multiples of 7: $sevenProduct")
println(s"Numbers even AND divisible by 3: ${evenAndDivBy3.mkString(", ")}")

## 🛑 Cleanup

**Always stop your SparkSession to free resources!**

In [ ]:
// Stop SparkSession
spark.stop()
println("🛑 Spark Application Stopped")
println("✅ All resources cleaned up!")

## 📚 What Next?

**🎉 Congratulations!** You've completed your first Spark application!

**You've learned:**
- ✅ Creating and configuring SparkSession
- ✅ Working with RDDs (Resilient Distributed Datasets)
- ✅ Understanding lazy evaluation and transformations vs actions
- ✅ Building word count applications
- ✅ Monitoring Spark applications
- ✅ Basic performance considerations

**Key Concepts Mastered:**
- **RDD**: Spark's core distributed data abstraction
- **Transformations**: Lazy operations (map, filter, flatMap)
- **Actions**: Eager operations that trigger computation (collect, count, sum)
- **Lazy Evaluation**: Operations are not executed until an action is called
- **Distributed Computing**: Data is automatically partitioned across cores/nodes

**Next Steps:**
1. Complete all exercises with your own implementations
2. Experiment with different RDD operations
3. Move to **02: RDD Mastery** for advanced transformations
4. Check out the **Spark UI** at http://localhost:4040 (if running locally)

**Help & Resources:**
- Stuck on exercises? Check the solutions in the `solutions/` directory
- Need help? Check the [official Spark documentation](https://spark.apache.org/docs/latest/)
- More examples? Look at the `examples/` directory

---

**Remember:** *"The best way to learn Spark is by doing Spark things!"* ⚡